In [106]:
import pandas as pd
import random
from collections import Counter
from pyDatalog import pyDatalog as plog

In [107]:
plog.clear()
df = pd.read_csv('All_Products_Data.csv')
df["Product_Id"] = range(1 , len(df)+1)
df.insert(0, "Product_Id", df.pop("Product_Id"))
df.head()

,Product_Id,Product Name,minprice,maxprice,Description,Categories,Image,Tags List
0,1,Haier 32 Inches 4K QLED Google TV H32S80EFX,48000,53000,The Haier 32S80EFX is a 32-inch Smart QLED LED...,"Small Electronics,LEDs & Audio",Haier_32_Inches_4K_QLED_Google_TV_H32S80EFX.jpg,"Energy-efficient,Smart,Bluetooth"
1,2,Haier 40 Inch FHD Android LED TV 40K800FX,64999,74999,The 40-inch Full HD Smart TV features Google T...,"Small Electronics,LEDs & Audio",Haier_40_Inch_FHD_Android_LED_TV_40K800FX.jpg,"LED,Energy-efficient,Bluetooth"
2,3,Haier 40 Inches 4K FHD Google TV H40K800FXS,69899,78999,The Haier H40K800FX is a Full HD Smart Google ...,"Small Electronics,LEDs & Audio",Haier_40_Inches_4K_FHD_Google_TV_H40K800FXS.jpg,"Smart,Wi-Fi,Connectivity"
3,4,Haier 43 Inches 4K Google QLED TV 43S800UX,90000,95000,Buy Haier 43 Inches 4K Google QLED TV 43S800UX...,"Small Electronics,LEDs & Audio",Haier_43_Inches_4K_Google_QLED_TV_43S800UX.jpg,"Sound,Bluetooth,LED"
4,5,Haier 50 Inches 4K Google QLED TV 50S800UX,127500,130750,Buy Haier 50 Inches 4K Google QLED TV 50S800UX...,"Small Electronics,LEDs & Audio",Haier_50_Inches_4K_Google_QLED_TV_50S800UX.jpg,"Smart,Audio,Energy-efficient"


In [108]:
plog.create_terms('SimilarCategory, SimilarTag , X , Y , P, Z , C , T , Product,Lies,upper_bound,lower_bound, HasPrice, User, HasCategory, HasTag, LikedByUser, Recommended , min_price')

In [109]:
df.columns

Index(['Product_Id', 'Product Name', 'minprice', 'maxprice', 'Description',
       'Categories', 'Image', 'Tags List'],
      dtype='object')

In [ ]:
def addFacts(user_data,df):
    # Facts for Products
    for index, row in df.iterrows():

        id = row['Product_Id']
        name = row['Product Name']
        min_price = row['minprice']
        max_price = row['maxprice']
        Categories = row['Categories']
        Categories = Categories.split(',')
        Tags = row["Tags List"]
        Tags = Tags.split(',')
        # print(Categories)
        +Product(id,name)
        +HasPrice(id,min_price)

        for category in Categories:
            +HasCategory(id,category)

        for tag in Tags:
            +HasTag(id , tag)
        
    #  For users, you can define likes using LikedByUser
    for index,row in user_data.iterrows():
        prod_num = row['prod_id']
        user_id = row['user_id']
        +LikedByUser(user_id, prod_num)


# Sample data to insert into the DataFrame
data = {
    'user_id': [1, 1, 2, 3, 3],
    'prod_id': [1, 14, 19, 42, 15]
    
}

user_data = pd.DataFrame(data)
addFacts(user_data , df)

In [120]:
def SimilarPrice(base_price):
    lower_bound = base_price * 0.8  # 20% less than base price
    upper_bound = base_price * 1.2  # 20% more than base price

    return plog.ask(f'Lies(X, P , {lower_bound} , {upper_bound})')

# result = get_price_range_products(20000)
# result.answers

In [ ]:

SimilarCategory(X , Y , Z) <= (LikedByUser(Z, X) & HasCategory(X, C) & HasCategory(Y, C) & (X != Y))
SimilarTag(X, Y , Z) <= (LikedByUser(Z, X) & HasTag(X, T)  & HasTag(Y, T) & (X != Y))
Lies(X, P , lower_bound , upper_bound) <= HasPrice(X , P) & (P > lower_bound) & (P < upper_bound)

Lies(X,P,'lower_bound','upper_bound') <= HasPrice(

In [122]:
def SimilarCategoryMethod(user_id, product_id):
    return plog.ask(f'SimilarCategory({product_id}, Y, {user_id})')

In [123]:
def SimilarTagMethod(user_id, product_id):
    return plog.ask(f'SimilarTag({product_id}, Y, {user_id})')

In [ ]:
# # plog.create_terms('Count') 
# SimilarCategory(X, Y, Z, C) <= (LikedByUser(Z, X) & HasCategory(X, C) & HasCategory(Y, C) & (X != Y))
# plog.ask(f'SimilarCategory({19}, Y , {1} , C)').answers

In [74]:
# len(HasCategory(1, C) & HasCategory(2, C))

In [75]:
# plog.ask(f'SimilarCategory({41}, Y , {1} , C)')
# SimilarCategory(41 , Y , 1 , C)

In [76]:
# HasPrice(1 , Y)[0][0]

In [ ]:
def recommend_products(user_id, top_n=10) :

    liked_products = plog.ask(f'LikedByUser({user_id}, Y)')
    product_similarity_dict = {}
    for product in liked_products.answers:
        prod_id = product[0]
        similar_product = SimilarCategoryMethod(user_id , prod_id) # plog.ask(f'SimilarCategory({prod_id}, Y , {user_id})')
        if similar_product is not None:
            for p in similar_product.answers:
                product_id = p[0]
                if product_id in product_similarity_dict:
                    product_similarity_dict[product_id] += (len(HasCategory(prod_id, C) & HasCategory(product_id, C))*10)
                else:
                    product_similarity_dict[product_id] = (len(HasCategory(prod_id, C) & HasCategory(product_id, C))*10)
        similar_product = None
        similar_product = SimilarTagMethod(prod_id , user_id) # plog.ask(f'SimilarTag({prod_id}, Y , {user_id})')
        if similar_product is not None:
            for p in similar_product.answers:
                product_id = p[0]
                if product_id in product_similarity_dict:
                    product_similarity_dict[product_id] += (len(HasTag(prod_id, C) & HasTag(product_id, C))*5)
                else:
                    product_similarity_dict[product_id] = ((len(HasTag(prod_id, C) & HasTag(product_id, C))*5))
        similar_product = None
        base_price = HasPrice(1 , Y)[0][0] 
        similar_product = SimilarPrice(base_price)
        if similar_product is not None:
            for p in similar_product.answers:
                product_id = p[0]
                if product_id in product_similarity_dict:
                    product_similarity_dict[product_id] += 3
                else:
                    product_similarity_dict[product_id] = 3
    list_of_tuples = list(product_similarity_dict.items())
    sorted_list = sorted(list_of_tuples, key=lambda x: x[1], reverse=True)
    print(sorted_list)
    new_list = [x[0] for x in sorted_list[:top_n]]
    return new_list

In [136]:
ls = LikedByUser(1 , Y)
ls

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
user_id = 1
products = recommend_products(user_id)
ys = LikedByUser(user_id , Y)
print(ys)
print(products)
print("Liked")
for y in ys:
    Product(y[0] , X)
    print(X)
print("Reffered")
for y in products:
    (Product(y , X))
    print(X)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# recommended_products = recommend_products(user_id=1)
# if recommended_products:
#     for product_id in recommended_products:
#         # Ensure the product_id exists in the DataFrame
#         product_name = df[df['Product_Id'] == product_id]
#         if not product_name.empty:
#             print(f"Recommended Product: {product_name['Product Name'].values[0]}")
#         else:
#             print(f"Product with ID {product_id} not found in the product list.")
# else:
#     print("No recommendations found.")

In [ ]:
from pyDatalog import pyDatalog
import pandas as pd
import json

# Create terms
pyDatalog.create_terms(
    "category, item, Smartphones, Laptops, Perfumes_and_Fragrances, Skincare, Food_Items, Home_Decor, Furniture, Women_s_Clothing, X, Y"
)

+category('iPhone 9', 'Smartphones')
+category('Samsung Universe 9', 'Smartphones')
+category('OPPOF19', 'Smartphones')
+category('Huawei P30', 'Smartphones')

+category('MacBook Pro', 'Laptops')
+category('Samsung Galaxy Book', 'Laptops')
+category('Microsoft Surface Laptop 4', 'Laptops')
+category('Infinix INBOOK', 'Laptops')
+category('HP Pavilion 15-DK1056WM', 'Laptops')

+category('perfume Oil', 'Perfumes_and_Fragrances')
+category('Brown Perfume', 'Perfumes_and_Fragrances')
+category('Fog Scent Xpressio Perfume', 'Perfumes_and_Fragrances')
+category('Non-Alcoholic Concentrated Perfume Oil', 'Perfumes_and_Fragrances')
+category('Eau De Perfume Spray', 'Perfumes_and_Fragrances')
+category('Fog Scent Xpressio Perfume custom', 'Perfumes_and_Fragrances')

+category('Hyaluronic Acid Serum', 'Skincare')
+category('Tree Oil 30ml', 'Skincare')
+category('Oil Free Moisturizer 100ml', 'Skincare')
+category('Skin Beauty Serum.', 'Skincare')
+category('Freckle Treatment Cream- 15gm', 'Skincare')

+category('Daal Masoor 500 grams', 'Food_Items')
+category('Elbow Macaroni - 400 gm', 'Food_Items')
+category('Orange Essence Food Flavour', 'Food_Items')
+category('Cereals muesli fruit nuts', 'Food_Items')
+category('Gulab Powder 50 Gram', 'Food_Items')

+category('Plant Hanger For Home', 'Home_Decor')
+category('Flying Wooden Bird', 'Home_Decor')
+category('3D Embellishment Art Lamp', 'Home_Decor')
+category('Handcraft Chinese style', 'Home_Decor')
+category('Key Holder', 'Home_Decor')

+category('Mornadi Velvet Bed', 'Furniture')
+category('Sofa for Coffee Cafe', 'Furniture')
+category('3 Tier Corner Shelves', 'Furniture')
+category('Plastic Table', 'Furniture')



def read_json_to_dataframe(file_path):
    """
    Reads a JSON file line by line, converts it to a DataFrame.
    Args:
        file_path (str): Path to the JSON file.
    Returns:
        pd.DataFrame: A pandas DataFrame containing the data.
    """
    data = []  # List to store processed records

    try:
        # Open the file and load the JSON data
        with open(file_path, 'r') as file:
            json_data = json.load(file)

        # Process the JSON data one by one
        for record in json_data:
            processed_record = {
                "_id": record["_id"],
                "title": record["title"],
                "description": record["description"],
                "price": record["price"],
                "discountPercentage": record["discountPercentage"],
                "stockQuantity": record["stockQuantity"],
                "brand": record["brand"],
                "category": record["category"],
                "thumbnail": record["thumbnail"],
                "images": record["images"],
                "isDeleted": record["isDeleted"],
                "updatedAt": record["updatedAt"]
            }
            data.append(processed_record)

        # Convert the list of dictionaries into a pandas DataFrame
        df = pd.DataFrame(data)
        return df

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return pd.DataFrame()  # Return an empty DataFrame
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return pd.DataFrame()

# Specify your JSON file path
file_path = 'productdetails.json'

# Call the function to read and convert the data
df1 = read_json_to_dataframe(file_path)


def display_category_products(product, df):
    """
    Displays the details of products related to the given product
    from DataFrame.
    """
    # Get the category of the selected product
    category_of_selected_item_result = category(product, Y)

    if category_of_selected_item_result:
        category_of_selected_item = category_of_selected_item_result[0]
        related_products = category(X, category_of_selected_item[0])
        related_product_names = [item[0] for item in related_products if item[0] != product] # Extract names except the selected product

        # Filter df1 to only contain related product details
        filtered_df = df[df['title'].isin(related_product_names)]
        if not filtered_df.empty:
          print(f"Details of products related to '{product}':")
          print(filtered_df)
        else:
          print(f"No details found for products related to '{product}'.")
    else:
        print(f"Product '{product}' not found in categories.")

product = "iPhone 9"
# Example Usage
display_category_products(product, df1)